In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

#Configuration
spark = SparkSession\
    .builder\
    .master('local[2]')\
    .appName('quake_etl')\
    .config('spark.jars.packages','org.mongodb.spark:mongo-spark-connector_2.12:2.4.1')\
    .getOrCreate() 
#load data
unemp = spark.read.csv(r"C:\Users\israt\OneDrive\Desktop\DM_Project\project\unemployment.csv", header=True)
unemp.show(5)

+--------------------+------------+--------------------+-----------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----------+-----------+-----------+----+----+----+----+-----------+----+----+-----------+----+----+-----------+----+----+-----------+-----------+----+----+-----------+----+----+-----------+----------+-----------+-----------+-----------+-----------+----+-----------+----+----+-----------+----+----+-----------+-----------+----+----+----+----+----+----+----+----+
|        Country Name|Country Code|      Indicator Name|   Indicator Code|1960|1961|1962|1963|1964|1965|1966|1967|1968|1969|1970|1971|1972|1973|1974|1975|1976|1977|1978|1979|1980|1981|1982|1983|       1984|       1985|       1986|1987|1988|1989|1990|       1991|1992|1993|       1994|1995|1996|       1997|1998|1999|       2000|       2001|2002|2003|       2004|2005|2006|       2007|      2008|       2009|       2010|       2011|       2012|

In [2]:
#delete column
delete_columns = ['Country Code','Indicator Name','Indicator Code','1960','1961','1962','1963','1964','1965','1966','1967','1968','1969','1970','1971','1972','1973','1974','1975','1976','1977','1978','1979','1980','1981','1982','1983',       '1984',       '1985'       ,'1986','1987','1988','1989','1990',       '1991','1992','1993'   ,    '1994','1995','1996',       '1997','1998','1999',       '2000'     ,  '2001','2002','2003',       '2004','2005','2006'  ,    '2007'  ,    '2008',       '2009',       '2010'   ,    '2011'       ,'2012','2013' ,      '2014','2015','2016'       ,'2017', '_c66','_c67','_c68','_c69','_c70','_c71','_c72','_c73']
unempl = unemp.drop(*delete_columns)
unempl.show(5)
unempl.printSchema()

+--------------------+----+----+-----------+-----------+
|        Country Name|2018|2019|       2020|       2021|
+--------------------+----+----+-----------+-----------+
|               Aruba|null|null|       null|       null|
|Africa Eastern an...|null|null|       null|       null|
|         Afghanistan|null|null|11.71000004|5.579999924|
|Africa Western an...|null|null|       null|       null|
|              Angola|null|null|       null|       null|
+--------------------+----+----+-----------+-----------+
only showing top 5 rows

root
 |-- Country Name: string (nullable = true)
 |-- 2018: string (nullable = true)
 |-- 2019: string (nullable = true)
 |-- 2020: string (nullable = true)
 |-- 2021: string (nullable = true)



In [3]:
#convert string to double
unemploy = unempl.withColumn('2018',unempl['2018'].cast(DoubleType())).withColumn('2019',unempl['2019'].cast(DoubleType())).withColumn('2020',unempl['2020'].cast(DoubleType())).withColumn('2021',unempl['2021'].cast(DoubleType()))
unemploy.printSchema()
unemploy.show(5)

root
 |-- Country Name: string (nullable = true)
 |-- 2018: double (nullable = true)
 |-- 2019: double (nullable = true)
 |-- 2020: double (nullable = true)
 |-- 2021: double (nullable = true)

+--------------------+----+----+-----------+-----------+
|        Country Name|2018|2019|       2020|       2021|
+--------------------+----+----+-----------+-----------+
|               Aruba|null|null|       null|       null|
|Africa Eastern an...|null|null|       null|       null|
|         Afghanistan|null|null|11.71000004|5.579999924|
|Africa Western an...|null|null|       null|       null|
|              Angola|null|null|       null|       null|
+--------------------+----+----+-----------+-----------+
only showing top 5 rows



In [4]:
#convert the all null values of the table into 0.0
unemployment=unemploy.na.fill(0)
unemployment.show(5)

+--------------------+----+----+-----------+-----------+
|        Country Name|2018|2019|       2020|       2021|
+--------------------+----+----+-----------+-----------+
|               Aruba| 0.0| 0.0|        0.0|        0.0|
|Africa Eastern an...| 0.0| 0.0|        0.0|        0.0|
|         Afghanistan| 0.0| 0.0|11.71000004|5.579999924|
|Africa Western an...| 0.0| 0.0|        0.0|        0.0|
|              Angola| 0.0| 0.0|        0.0|        0.0|
+--------------------+----+----+-----------+-----------+
only showing top 5 rows



In [5]:
# connect with mongodb by creating database

unemp.write.format('mongo')\
    .mode('overwrite')\
    .option('spark.mongodb.output.uri','mongodb://127.0.0.1/Unemployment.unemp').save()

unempl.write.format('mongo')\
    .mode('overwrite')\
    .option('spark.mongodb.output.uri','mongodb://127.0.0.1/Unemployment.unempl').save()


unemploy.write.format('mongo')\
    .mode('overwrite')\
    .option('spark.mongodb.output.uri','mongodb://127.0.0.1/Unemployment.unemploy').save()


unemployment.write.format('mongo')\
    .mode('overwrite')\
    .option('spark.mongodb.output.uri','mongodb://127.0.0.1/Unemployment.unemployment').save()